In [1]:
import spacy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json

%matplotlib inline

# Import the review data

In [26]:
#The directory where the data is located
path = "C:/Users/Andy Chan/desktop/INT15/project/data/yelp_dataset_1/"
#The filename of the data to be loaded
filename = "review.json"

#Read the review daat
review = []
with open(path+filename,encoding="utf-8") as json_file:
    for i in json_file:
        review.append(json.loads(i))
#Convert it to a data frame
review_df = pd.DataFrame(review)
review_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


**This is the review dataset for all review, including the businesses I excluded, so I will filter them by business_id.**

In [26]:
#Import the processed business detail 
biz_df = pd.read_csv("processed_details.csv",encoding='unicode_escape')
#Get the business ids
biz_id = biz_df.business_id

**Filter out the reviews with their business_ids.**

In [33]:
#Get the index of the review that matches the ids
matched_idx = [i for i in range(len(review_df)) if (review_df.loc[i,"business_id"] in list(biz_id))]
#Use the index to filter
review_df = review_df.loc[matched_idx]

In [27]:
#See how many reviews matched
len(review_df)

6685900

In [46]:
#Drop "cool","funny","useful","user_id"
#review_df = review_df.drop(columns=["funny","cool","useful","user_id"])
#Ouput a filterred reivew dataset
review_df.to_csv("filtered_review.csv",index=False)

# Load the filtered review, and start the pre-processing

In [5]:
review_df = pd.read_csv("filtered_review.csv")
review_df = review_df.sample(frac=0.1,random_state=314)

## Text Cleaning


In [6]:
text = review_df["text"]

### Lowercase

In [7]:
text = text.str.lower()

### Expand Contractions

In [8]:
#Define a list of contraction
contractions=["n't","'ll","'ve","'re", "'d","'s"]
#Define a list of expaned contraction
expanded=[" not"," will"," have"," are"," would",""]

for i in range(len(expanded)):
    text = text.str.replace(contractions[i],expanded[i])

### Remove numbers

In [9]:
text = text.str.replace(r"[0-9]","")

### Remove special characters and punctuation

In [10]:
#Remove punctuation
text = text.str.replace(r'[^\s\w]', " ")

### Strip extra white space

In [11]:
text= text.str.replace(r"[ ]{2,}", " ") \
                .str.strip()

## Remove stopwords using Spacy

In [10]:
#!pip --install spacy
#!python -m spacy download en_core_web_md


In [12]:
import spacy

#Load Spacy model
nlp = spacy.load("en_core_web_md", parser=False, entity=False)

#Load stopwords
stopwords = spacy.lang.en.stop_words.STOP_WORDS


In [13]:
#Tokenization
tokens = text.apply(lambda r:nlp(r))

In [ ]:
gg


In [25]:
tokens.iloc[0][10].lemma_

'price'

In [71]:
text[0]

'walked in around on a friday afternoon we sat at a table just off the bar and walked out after min or so do not even think they realized we walked in however everyone at the bar noticed we walked in service was non existent at best not a good way for a new business to start out oh well the location they are at has been about different things over the past several years so they will just be added to the list smdh'

In [68]:
tt= text.copy().str.strip()
tt[0]

'walked in around on a friday afternoon we sat at a table just off the bar and walked out after min or so do not even think they realized we walked in however everyone at the bar noticed we walked in service was non existent at best not a good way for a new business to start out oh well the location they are at has been about different things over the past several years so they will just be added to the list smdh'